In [3]:
import feedparser
import spacy
import pandas as pd
import sqlite3
from geopy.geocoders import Nominatim

### Podcast Feed Scraping
Download the entire Podcast feed and transform into DataFrame for easier analysis

In [137]:
feed_url = "https://geschichten-aus-der-geschichte.podigee.io/feed/mp3"
episode_feed = feedparser.parse(feed_url)

df = pd.DataFrame(episode_feed['entries'])
df = df[['title', 'subtitle', 'summary', 'published', 'link']]

In [138]:
df

,title,subtitle,summary,published,link
0,"FGAG07: Bibi & Tina, Casinos in Macau und noch...","Ein FeedGAG u.a über Galla Placidia, Glücksspi...",Wir sprechen in diesem FeedGAG u.a. über Gall...,"Sun, 27 Nov 2022 12:00:00 +0000",https://www.geschichte.fm/archiv/fgag07/
1,GAG374: Ludwik Fleck und das Fleckfieber,Eine Geschichte über die Bekämpfung einer Kran...,"Das Fleckfieber wurde nicht nach ihm benannt, ...","Wed, 23 Nov 2022 08:20:00 +0000",https://www.geschichte.fm/archiv/gag374/
2,GAG373: Morocco und der Kluge Hans,Eine Geschichte über zwei außergewöhnliche Pferde,Wir springen in dieser Folge in zwei unterschi...,"Wed, 16 Nov 2022 08:45:00 +0000",https://www.geschichte.fm/archiv/gag373/
3,GAG372: Wie das Roulette eine Null verlor,Eine Geschichte über einen Börsenbetrug und Sp...,Im Jahr 1834 finden die Brüder Blanc einen Weg...,"Wed, 09 Nov 2022 08:15:00 +0000",https://www.geschichte.fm/archiv/gag372/
4,GAG371: Galla Placidia,Eine Geschichte über die mächtigste Frau der S...,Wir springen diesmal ins Rom des 5. Jahrhunder...,"Wed, 02 Nov 2022 08:45:00 +0000",https://www.geschichte.fm/archiv/gag371/
...,...,...,...,...,...
388,"GAG04: Wellingtons Rache, oder: Ein Bein für e...","Eine Geschichte über ein Bein, das zu einer To...",Wir springen in die Zeit der napoleonischen Kr...,"Sun, 18 Oct 2015 19:27:16 +0000",https://www.geschichte.fm/podcast/zs04
389,GAG03: Fitness-Parcours für den Pharao,"Eine Geschichte über Pharaonen, die ihre Regie...","Wer regieren will, muss fit sein? Aber wie läs...","Sun, 11 Oct 2015 17:30:33 +0000",https://www.geschichte.fm/podcast/zs03
390,GAG02: Tatortschau mit Wiedererkennungswert,Eine Geschichte über Fingerabdrücke und die An...,Wir springen diesmal nicht ganz so weit zurück...,"Mon, 05 Oct 2015 10:19:27 +0000",https://www.geschichte.fm/podcast/zs02
391,GAG01: Vier Langobarden-Könige und ein Trinkbe...,Eine Geschichte über den Langobardenkönig Albo...,Frühes Mittelalter in Italien: Times are rough...,"Thu, 01 Oct 2015 09:07:13 +0000",https://www.geschichte.fm/podcast/zs01


### Database Helper Class
Class able to connect to Flask SQLite Database and helper function for querying and writing 

In [145]:
from datetime import datetime

class DB():
    con = sqlite3.connect("database.db")
    cur = con.cursor()
    
    def save_episode(self, id, title, summary, published, link):
        query = f"INSERT INTO 'episode' (id, title, summary, published, link) VALUES ('{id}', '{title}', '{summary}', '{datetime.strptime(published, '%a, %d %b %Y %H:%M:%S %z')}', '{link}')"
        self.cur.execute(query)
        self.con.commit()
    
    def save_location(self, episode_id, name, longitude, latitude):
        query = f"INSERT INTO 'location' (episode_id, name, longitude, latitude) VALUES ('{episode_id}', '{name}', '{longitude}', '{latitude}');"
        self.cur.execute(query)
        self.con.commit()
    
    def get_all_episodes(self):
        query = "SELECT * FROM 'episode' ORDER BY 'id' DESC;"
        return self.cur.execute(query).fetchall()
    
    def get_episode(self, id):
        query = f"SELECT * FROM 'episode' WHERE id='{id}';"
        return self.cur.execute(query).fetchone()
        
    def describe_table(self, table):
        return self.cur.execute(f"SELECT sql FROM sqlite_schema WHERE name='{table}';").fetchall()

In [130]:
db = DB()
# db.save_episode('ZZZ999', 'title', 'summary', 'Dec 21 2020  1:01PM', 'link.com')
# db.save_location('ZZZ999', 'Pari', '123', '321')
# db.describe_table('episode')
# print(db.get_episode('ZZZ9999'))
db.get_all_episodes()

[]

### Get Location Data
Extract all location information in podcast summary using spaCys NER. Using geopy to query OSM API for coordinates.
Write Information into DB.

In [147]:
from tqdm.notebook import tqdm

nlp = spacy.load("de_core_news_sm")
geolocator = Nominatim(user_agent="karten_aus_der_geschichte")

db = DB()

for episode in episode_feed['entries']:
    title = episode['title']
    symbol = title.split(':')[0]
    summary = episode['summary']
    link = episode['link']
    published = episode['published']
    
    
    db.save_episode(symbol, title, title, published, link)
    
    summary_doc = nlp(summary)
    
    for word in summary_doc.ents:
        if word.label_ == 'LOC' and 'http' not in word.text:
            location = geolocator.geocode(word.text, language="de")
            if(location):
                db.save_location(symbol, word.text, location.longitude, location.latitude)

  0%|          | 0/393 [00:00<?, ?it/s]

INSERT INTO 'episode' (id, title, summary, published, link) VALUES ('FGAG07', 'FGAG07: Bibi & Tina, Casinos in Macau und noch mehr Schiffe in Maisfeldern', 'FGAG07: Bibi & Tina, Casinos in Macau und noch mehr Schiffe in Maisfeldern', '2022-11-27 12:00:00+00:00', 'https://www.geschichte.fm/archiv/fgag07/')
INSERT INTO 'episode' (id, title, summary, published, link) VALUES ('GAG374', 'GAG374: Ludwik Fleck und das Fleckfieber', 'GAG374: Ludwik Fleck und das Fleckfieber', '2022-11-23 08:20:00+00:00', 'https://www.geschichte.fm/archiv/gag374/')
INSERT INTO 'episode' (id, title, summary, published, link) VALUES ('GAG373', 'GAG373: Morocco und der Kluge Hans', 'GAG373: Morocco und der Kluge Hans', '2022-11-16 08:45:00+00:00', 'https://www.geschichte.fm/archiv/gag373/')
INSERT INTO 'episode' (id, title, summary, published, link) VALUES ('GAG372', 'GAG372: Wie das Roulette eine Null verlor', 'GAG372: Wie das Roulette eine Null verlor', '2022-11-09 08:15:00+00:00', 'https://www.geschichte.fm/arc